# Análisis Espectral de Sonidos de Pájaros Europeos

In [ ]:
#Imports Globales

import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# Funciones Globales.
def count_files_in_subfolders(main_folder):
    sfcounts = {}
    # root is the current directory path
    # dirs is a list of subdirectories in the current directory
    # files is a list of files in the current directory
    for root, dirs, files in os.walk(main_folder):
        # Count only in subfolders, not in the main folder itself
        if root != main_folder:
            h5_files = [file for file in files if file.endswith(".h5")]
            subfolder_name = os.path.basename(root)
            sfcounts[subfolder_name] = len(h5_files)
    return sfcounts

def plot_histogram(sub_folder_counts):
    # Extract counts and subfolder names
    sorted_subfolders = sorted(sub_folder_counts.items(), key=lambda x: x[1], reverse=True)

    # Extract counts and subfolder names after sorting
    counts = [count for subfolder, count in sorted_subfolders]
    subfolders = [subfolder for subfolder, count in sorted_subfolders]

    # Plot histogram
    plt.figure(figsize=(8, 6))
    bars = plt.bar(subfolders, counts, color='mediumturquoise', edgecolor='lightseagreen')
    plt.xlabel('Bird Names')
    plt.ylabel('Number of Spectograms')
    plt.title('Number of Spectograms of Birds', weight='bold')
    plt.xticks(rotation=45, ha='right')

    # Add counts above each bar
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count),
                 ha='center', va='bottom')
    plt.tight_layout()
    plt.show()

## Counting number of spectograms for each bird name

In [ ]:
# Ruta a los espectrogramas convertidos a hdf5
main_folder_path = os.path.abspath("./data/spects_h5")

# Conteo de archivos por tipo de pájaro
subfolder_counts = count_files_in_subfolders(main_folder_path)

# Histograma de distribución de muestras
plot_histogram(subfolder_counts)

In [ ]:
# Cargar los espectrogramas & labels
spectrograms_folder = os.path.abspath("./data/spectrograms")
bird_folders = [folder for folder in os.listdir(spectrograms_folder) if not os.path.isfile(folder)]
print(bird_folders)

# Crear un archivo h5 para agrupar los espectrogramas por nombre de audio
with h5py.File(os.path.abspath("./data/spectrograms.h5"), "w") as hf:
    for bird_folder in bird_folders:
        bird_name = bird_folder.split("_")[0]  # Obtener el nombre del pájaro
        spectogram_files = os.listdir(os.path.join(spectrograms_folder, bird_folder))
        for spectogram_file in spectogram_files:
            if spectogram_file.endswith(".npy"):
                audio_name = spectogram_file.split("_")[0]
                if audio_name not in hf:
                    audio_group = hf.create_group(audio_name)
                    # Agregar el nombre del pájaro al grupo del audio
                    audio_group.attrs["bird"] = bird_name
                spectogram_path = os.path.join(spectrograms_folder, bird_folder, spectogram_file)
                spectogram = np.load(spectogram_path)
                hf[audio_name].create_dataset(spectogram_file, data=spectogram)
                
# Open the h5 file
audio_names = []
with h5py.File(os.path.abspath("./data/spectrograms.h5")) as hf:
    # Get the keys of the groups (audio names)
    audio_names = list(hf.keys())

print(audio_names)

In [ ]:
# Empty list to store the name of each group and bird name
bird_info = []
# Leer el archivo H5 existente
with h5py.File(os.path.abspath("./data/spectrograms.h5")) as hf:
    for audio_name in hf.keys():
        bird_name = hf[audio_name].attrs["bird"]
        bird_info.append([audio_name, bird_name])

print(bird_info)

In [ ]:
from collections import defaultdict

audios_per_bird = defaultdict(list)
for audio in bird_info:
    bird_name = audio[1]
    audios_per_bird[bird_name].append(audio[0])

train_audio_names = []
test_audio_names = []

for bird, audios in audios_per_bird.items():
    num_audios = len(audios)
    num_train = int(0.7 * num_audios)

    train_audio_names.extend([audio for audio in audios[:num_train]])
    test_audio_names.extend([audio for audio in audios[num_train:]])

print("Training list:")
print(train_audio_names)

print("Testing list:")
print(test_audio_names)

In [ ]:
# Create a new file h5 to store the data for training the model
with (  h5py.File(os.path.abspath("./data/data_train.h5"), "w") as hf_train,
        h5py.File(os.path.abspath("./data/spectrograms.h5")) as hf ):
    # Create group "X" to store the spectograms
    X_group = hf_train.create_group("X")

    # Create group "y" to store the names of the birds (labels)
    y_group = hf_train.create_group("y")

    for audio_name in train_audio_names:
        # get the name of the corresponding bird
        bird_name = hf[audio_name].attrs["bird"]

        # read the spectograms of the audio
        spectrogram_names = list(hf[audio_name].keys())

        for spectrogram_name in spectrogram_names:
            # Read the espectogram
            spectrogram_data = hf[audio_name][spectrogram_name][...]
            # Save the espectogram in group X
            X_group.create_dataset(spectrogram_name, data=spectrogram_data)
            # Save the name of the corresponding bird in th group Y
            y_group.create_dataset(spectrogram_name, data=bird_name)
            
# create the LabelEncoder
le = LabelEncoder()
X_train = []
y_train = []

# Agrupamos espectrogramas
with h5py.File(os.path.abspath("./data/data_train.h5")) as hf_train:
    # Access to a specific group
    group_Xtrain = 'X'
    group_ytrain = 'y'
    groupXtrain = hf_train[group_Xtrain]
    groupytrain = hf_train[group_ytrain]

    # Apilamos espectrogramas y labels
    for key in groupXtrain.keys():
        spectogram = groupXtrain[key]
        X_train.append(spectogram[()])

    for key in groupytrain.keys():
        bird = groupytrain[key]
        y_train.append(bird[()])


In [ ]:
y_train = [data.decode('utf-8') for data in y_train]
y_train

In [ ]:
print(len(X_train))
print(X_train[0])
print(len(y_train))
print(y_train[0])

# Encode the labels
y_train = le.fit_transform(y_train)
print(y_train)
# Print the labels of the numeric values
print(le.classes_)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = X_train / 255.0

X_train = X_train.reshape((-1, 224, 224, 1))
print(X_train.shape)

In [ ]:
print(y_train.shape)
print(len(y_train))

In [ ]:
from keras.src.losses import SparseCategoricalCrossentropy

# CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(20)    
])

# Compile the model
model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=35)


In [ ]:
model.save(os.path.abspath("./data/DL_model_birds.h5"))

In [ ]:
from tensorflow.keras.models import load_model 

# Load the model
loaded_model = load_model(os.path.abspath("./data/DL_model_birds.h5"))

## Testing part: X_test, y_test

In [ ]:
# H5 file to store the testing data
with (  h5py.File(os.path.abspath("./data/data_test.h5"), "w") as hf_test, 
        h5py.File(os.path.abspath("./data/spectrograms.h5")) as hf ):
    # Group "X" to store the spectograms for testing
    X_group = hf_test.create_group("X")

    #  Group "X" to store the labels for testing
    y_group = hf_test.create_group("y")

    for audio_name in test_audio_names:
        # Get the name of the corresponding bird of the audio
        bird_name = hf[audio_name].attrs["bird"]

        # read the spectograms
        spectrogram_names = list(hf[audio_name].keys())

        for spectrogram_name in spectrogram_names:
            # Read the spectograms
            spectrogram_data = hf[audio_name][spectrogram_name][...]
            # Save the spectogram in group "X"
            X_group.create_dataset(spectrogram_name, data=spectrogram_data)
            # Save the name of the corresponding bird in group "y"
            y_group.create_dataset(spectrogram_name, data=bird_name)

In [ ]:
# LabelEncoder
le = LabelEncoder()
X_test = []
y_test = []

In [ ]:
with h5py.File(os.path.abspath("./data/data_test.h5")) as hf_test:
    # Access to a specific group
    group_Xtest = 'X'
    group_ytest = 'y'
    groupXtest = hf_test[group_Xtest]
    groupytest = hf_test[group_ytest]

    # Save spectograms and labels
    for key in groupXtest.keys():
        spectogram = groupXtest[key]
        X_test.append(spectogram[()])

    for key in groupytest.keys():
        bird = groupytest[key]
        y_test.append(bird[()])

In [ ]:
y_test = le.fit_transform(y_test)
print(y_test)

print(len(X_test))
print(X_test[0])
print(len(y_test))
print(y_test[0])

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

X_test = X_test / 255.0
X_test = X_test.reshape((-1, 224, 224, 1))

In [ ]:
# Evaluation of the model with the testing data
loss, accuracy = model.evaluate(X_test, y_test)

print("Pérdida:", loss)
print("Precisión:", accuracy)

## Predictions of the models and confusion matrix

In [ ]:
# Prediction of the model with testing data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Assuming 'class_names' is a list containing your class labels
# class_names = ["Acrocephalus arundinaceus", "Acrocephalus melanopogon", "Acrocephalus scirpaceus",
#                "Alcedo atthis", "Anas platyrhynchos", "Anas strepera", "Ardea purpurea",
#                "Botaurus stellaris", "Charadrius alexandrinus", "Ciconia ciconia",
#                "Circus aeruginosus", "Coracias garrulus", "Dendrocopos minor",
#                "Fulica atra", "Gallinula chloropus", "Himantopus himantopus",
#                "Ixobrychus minutus", "Motacilla flava", "Porphyrio porphyrio", "Tachybaptus ruficollis"]

from utils import CLASS_NAMES
conf_matrix_df = pd.DataFrame(cm, index=CLASS_NAMES, columns=CLASS_NAMES)

# Plot
plt.figure(figsize=(10,7))
sns.heatmap(conf_matrix_df, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title("Confusion Matrix - Deep Learning Model")
plt.show()